In [23]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import pandas as pd
# import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('train.csv', sep=';')
bins = [0,90,100,110, 120, 130, 140, 150, 160, 170, 180, 250, 100000]
df['ap_hi'] = pd.cut(df['ap_hi'].values, bins, labels=False)
lo_bins = [0,40,50,60,70,80,90,100,110,120,100000]
df['ap_lo'] = pd.cut(df['ap_lo'].values, lo_bins, labels=False)
df['ap_hi'].fillna(3, inplace = True)
df['ap_lo'].fillna(4, inplace = True)

In [3]:
def preprocess(X, squares = False):
    xlen = X.shape[0]
    imb = X[:,3] / ((X[:,2]/100.0)**2)
    imb = imb.reshape((len(imb),1))
    X = np.hstack((X, imb))
    X[:,0] = (X[:,0].astype(int) / int(365)).astype(int) #number of years instead of days
    gender = X[:,1].astype(int) - 1
    cholesterol = X[:,6].astype(int) - 1
    gluc = X[:,7].astype(int) - 1
    ap_hi = X[:,4].astype(np.int64)
    ap_lo = X[:,5].astype(np.int64)
    gender_oh = np.zeros((xlen, 2)).astype(int)
    cholesterol_oh = np.zeros((xlen, 3)).astype(int)
    gluc_oh = np.zeros((xlen, 3)).astype(np.int64)
    ap_hi_oh = np.zeros((xlen,12)).astype(np.int64)
    ap_lo_oh = np.zeros((xlen,10)).astype(np.int64)
    gender_oh[np.arange(xlen), gender[np.arange(xlen)] - 1] = 1
    cholesterol_oh[np.arange(xlen), cholesterol[np.arange(xlen)] - 1] = 1
    gluc_oh[np.arange(xlen), gluc[np.arange(xlen)] - 1] = 1
    ap_hi_oh[np.arange(xlen), ap_hi[np.arange(xlen)]] = 1
    ap_lo_oh[np.arange(xlen), ap_lo[np.arange(xlen)]] = 1
    X = np.hstack((X, gender_oh))
    X = np.hstack((X, cholesterol_oh))
    X = np.hstack((X, gluc_oh))
    X = np.hstack((X, ap_hi_oh))
    X = np.hstack((X, ap_lo_oh))
    if (squares):
        counter = 0
        XX = np.zeros((X.shape[0], int((X.shape[1] * (X.shape[1] + 1)) / 2) + X.shape[1]))
        for i in range(X.shape[1]):
            for j in range(i + 1):
                XX[:,counter] = X[:,i] * X[:,j]
                counter = counter + 1
        XX[:,counter:] = X
        X = XX;
    return X

In [4]:
X_short = df.values[:,1:-4]
X_all = df.values[:,1:-1]
y = df.values[:,-1]
indexes = list(range(len(y)))
random.shuffle(indexes)
X_short = X_short[indexes]
X_all = X_all[indexes]
y = y[indexes]
X_short = preprocess(X_short, squares=False)
X_all = preprocess(X_all, squares=False)

In [5]:
X_short_train, X_short_test, y_short_train, y_short_test = train_test_split(X_short, y, test_size=0.3)
X_all_train, X_all_test, y_all_train, y_all_stest = train_test_split(X_all, y, test_size=0.3)

In [28]:
clf = BernoulliNB()
clf.fit(X_all_train, y_all_train.astype(np.int64))
y_pred = clf.predict_proba(X_all_test)[:,1]
print(roc_auc_score(y_all_stest, y_pred))

0.766264929908
